In [1]:
import csv, random, string
import numpy as np
from os import mkdir, rmdir
from shutil import move
from os.path import exists
from kmc import OrthogonalLattice, KineticMonteCarlo, InitialEnergies, SaddleEnergies

In [2]:
# Read in the data
with open('../csv/1.csv', 'r') as csvFile:
    reader = csv.reader(csvFile)
    rows = [row for row in reader]

In [3]:
# Get the energies
init_energies = []
saddle_energies = []
for i in range(len(rows)):
    if i == 0:
        continue
    init_energies.append(float(rows[i][2]))
    saddle_energies.append(float(rows[i][3]))
init_energies = np.array(init_energies)
saddle_energies = np.array(saddle_energies)
def reject_outliers(data, m = 100.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else np.zeros(len(d))
    return data[s<m]
init_energies = reject_outliers(init_energies)
saddle_energies = reject_outliers(saddle_energies)

In [4]:
init_params = (np.mean(init_energies), np.std(init_energies))
saddle_params = (np.mean(saddle_energies), np.std(saddle_energies))
print(f'I: {init_params}')
print(f'S: {saddle_params}')

In [5]:
# Lattice params
lattice_type = OrthogonalLattice.LatticeType.SC
lattice_size = [5, 5, 5, 5]
lattice_spacing = [1, 1, 1, 1]

In [6]:
# kmc params
run_iterable = range(3) # This can be a range or a list or whatever
steps = 1_000
temp = 800
output_folder = f'../dumps/'
name_prefix = 'run_'
name_postfix = '_4d_vacancy.dump'
work_folder = '/tmp/' # This is where the temporary files will be stored
compress = False # Whether to lzma compress the output files
# The output files will look like: name_prefix + run_iterable[i] + name_postfix

In [7]:
myLat = OrthogonalLattice(lattice_type, lattice_size, lattice_spacing)
random_folder_name = ''
while True:
    # Random folder in /tmp/ with 16 characters
    # We want this because /tmp/ is usually a RAM disk and writing to it is faster, ESPECIALLY if we are writing to multiple files simultaneously like in this case
    random_folder_name = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16))
    if not exists(f'{work_folder}{random_folder_name}'):
        mkdir(f'{work_folder}{random_folder_name}')
        break

file_names = [f'{work_folder}{random_folder_name}/{i}.dump' for i in run_iterable]
initial_energies = InitialEnergies(myLat.num_sites, np.random.normal, init_params)
saddle_energies = SaddleEnergies(myLat.get_adjacency_matrix(), np.random.normal, saddle_params, init_energies=initial_energies)
myKMC = KineticMonteCarlo(myLat, initial_energies, saddle_energies,temp)
myKMC.run_many(steps, file_names)
if not exists(output_folder):
        mkdir(output_folder)
for i in run_iterable:
    move(f'{work_folder}{random_folder_name}/{i}.dump', f'{output_folder}{name_prefix}{i}{name_postfix}')
# Delete the folder
rmdir(f'{work_folder}{random_folder_name}')